# Exploring Library Usage in San Francisco

## Project Start Date: 3/8/2018

## Analyst: Ted Petrou

## Summary
In this notebook, we will follow the outline from the **First Look at Data** section onwards. We will systematically explore the basic facets of the data. This is just a starting point for our analysis and will be used as a launching point for future investigation.

## Load libraries
The vast majority (or sometimes all) the analysis for this routine can be completed with just pandas. NumPy can help with a few rare cases where no available pandas method exists or where increase in performance is necessary.

Pandas can also make a few, but very common, visualizations. Seaborn is imported to make a few different and more aethstetically pleasing visualizations.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
library = pd.read_csv('data/library_usage.csv')

## First glance at data
Use the `head` or `tail` commands to display the first/last rows of data. JupyterLab now has the ability to display csv files, so you can permanently keep the dataset open to browse at your liking while simultaneously working in the notebook.

In [ ]:
library.head()

## Studying the data dictionary
Its important to take a close look at the data dictionary to get a solid understanding of each column.

## What if there is no data dictionary?
It is your job to find one or create one. I suggest asking database admins or searching online before attempting to create one yourself.

## Do you upkeep the data dictionary by hand or in pandas?
Pandas is not specifically designed for keeping notes in cells but it can be done somewhat pain-free. Pandas does however, allow you to add metadata very easily to columns. I will use pandas to keep all the notes and metadata. Using a spreadsheet is also acceptable and allows you to make edits a bit faster.

## Loading the data dictionary as a DataFrame
I converted the original data dictionary as an Excel file to a csv so that I could easily load it into pandas.

In [ ]:
library_dd = pd.read_csv('data/Data Dictionary.csv')
library_dd

In [ ]:
pd.options.display.max_colwidth

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
library_dd

## Pandas data types
The data types in the official data dictionary are generic and not specific to pandas. Instead of using this default data dictinoary, we can create our own. Let's first copy it into a new variable.

In [ ]:
library_dd_pandas = library_dd.copy()

## Replace the generic data types with pandas data types

In [ ]:
library.dtypes

In [ ]:
library_dd_pandas['Data Type'] = library.dtypes

In [ ]:
library_dd_pandas

## What went wrong?
Pandas aligns first on the index (the labels on the left in **bold**) of each object. `library.dtypes` has the **column names** as the index. The data dictionary just has **integers**. No alignment happens and missing values are the result.

## Move column name into the index
We can move the column names into the index from **`library_dd_pandas`** so that all future column data is automatically aligned properly.

In [ ]:
library_dd_pandas = library_dd_pandas.rename(columns={'Field Name': 'Column Name'}) \
                                     .set_index('Column Name')
library_dd_pandas

## Redo previous operation
Now that the indexes are identical, the alignment will 

In [ ]:
library_dd_pandas['Data Type'] = library.dtypes

In [ ]:
library_dd_pandas

## There's still something wrong - Birth Date column has a missing data type!
Each column's data type should never be missing, yet we are getting a missing value for the **`Birth Date`** column. This column is named **`Age Range`** in the actual dataset.

## Ensure all columns are the same in both DataFrames
Let's do a sanity check and see if there are same number of columns in the data as there are descriptions in the data dictionary.

In [ ]:
len(library.columns)

In [ ]:
len(library_dd_pandas.index)

## Failed sanity test
We don't even have the same number of values in each one.

## Find which values are in one but not the other
Use the `isin` method to find which values are in the data dictionary but are not columns in the data and vice-versa:

In [ ]:
isin_dd = library_dd_pandas.index.isin(library.columns)
library_dd_pandas.index[~isin_dd]

In [ ]:
isin_data = library.columns.isin(library_dd_pandas.index)
library.columns[~isin_data]

## Conclusions of column name mistakes
* **`Birth Date`** from the data dictionary is named **`Age Range`** in the data.
* **`Circulation Active Month`** is in the data but absent from the data dictionary.

Let's rename **`Birth Date`** in data dictionary

In [ ]:
library_dd_pandas = library_dd_pandas.rename(index={'Birth Date': 'Age Range'})
library_dd_pandas

## Insert new row for Circulation Active Month
This is tricky and not easily accomplished with pandas. 
* First, select the **`Circulation Active Year`** row as a **`Series`**
* Change the the definition and change the **`name`** which will be used as the new index label.

In [ ]:
circulation_am = library_dd_pandas.loc['Circulation Active Year']
circulation_am['Definition'] = 'Month the patron last checked out library materials, ' \
                               'or last logged into the library’s subscription databases ' \
                               'from a computer outside the library.'
circulation_am.name = 'Circulation Active Month'

## Cut old data dictionary to pieces and splice in new row

In [ ]:
df1 = library_dd_pandas.loc[:'Home Library Definition']
df2 = library_dd_pandas.loc['Circulation Active Year':]
library_dd_new = df1.append(circulation_am).append(df2)
library_dd_new

## Are we finally ready to align the data?
Let's find out one more time and add in the data types

In [ ]:
library_dd_new['Data Type'] = library.dtypes
library_dd_new

## Comparing the pandas data types to the original data types
All the data types seem to match. Pandas uses the **object** data type to hold text. But, if you look closely, the **`Circulation Active Year`** is text even though it appears to be an integer. We will come back to this later.

## Making notes and updating the data dictionary
Its quite straightforward to update the data dictionary. We just need to select the exact cell of data with the **`.loc`** indexer and give it a new value.

Let's make a note that the **`Circulation Active Month`** was not in the orginal data dictionary

In [ ]:
# rename column Notes (optional) to Notes
library_dd_new = library_dd_new.rename(columns={'Notes (optional)': 'Notes'})

In [ ]:
library_dd_new.loc['Circulation Active Month', 'Notes'] = 'This column was not in original data dictionary'
library_dd_new

## Continuing on with checklist
Find the number of rows and columns

In [ ]:
library.shape

## Get a distribution of the data types

In [ ]:
library.dtypes.value_counts()

## Find the number of missing values per column

In [ ]:
library.isna().sum()

## Append this as a column to the data dictionary
Index alignment helps us out here

In [ ]:
library_dd_new['Missing Values Count'] = library.isna().sum()
library_dd_new

## Can also do % of missing values

In [ ]:
library_dd_new['Missing Values %'] = library.isna().mean()
library_dd_new

## Is the data tidy?
This is an important question to ask towards the very beginning of a data analysis. Hadley Wickham is famous for instilling this core tennet.

Tidy data is when:
* Each variable forms a column
* Each observation forms a row
* Each observational unit forms a table

## Check to see if our data is tidy
Most data from relational databases will meet this criteria. This data is likely stored in a relational database and from the description we know that each row reprsents a single patron (an observation).

Some columns appear to have duplicate information and we may not need them. Although, this isn't strictly mentioned in the three tidy principles, duplicate information won't add anything to our analysis.

## Check for duplication of Patron Type Code amd Patron Type Description
We can do a **two-way contingency table** to count the co-occurrences of each pair to determine if there is indeed a one-to-one mapping

In [ ]:
patron_table = pd.crosstab(library['Patron Type Code'], library['Patron Type Definition'])
patron_table

# Ensuring a one to one relationship
Let's see if the sum equals the maximum for all columns. If it does, then we can drop one of the columns.

In [ ]:
patron_table.sum() == patron_table.max()

# Drop the Patron Type Code

In [ ]:
library_new = library.drop(columns='Patron Type Code')
library_new.head()

## Repeat for Notice Preference Code and Notice Preference Definition

In [ ]:
pd.crosstab(library['Notice Preference Code'], library['Notice Preference Definition'])

We can easily determine that there is indeed a one-to-one mapping. Let's again drop the code and keep the definition.

In [ ]:
library_new = library_new.drop(columns='Notice Preference Code')
library_new.head()

### Consolidating the dates
**`Circulation Active Month`** and **`Circulation Active Year`** may be consolidated to a single field. We can always strip out the componenets later if we need them.

Before we do this, let's take a look at the unique values/counts of each of these columns.

In [ ]:
library['Circulation Active Month'].value_counts()

In [ ]:
library['Circulation Active Year'].value_counts()

## Missing values?
It appears there are missing values **'None'**, but our **`isna`** method didn't catch them. Let's look at the unique values to get a closer look.

In [ ]:
library['Circulation Active Month'].unique()

## Date Parsing
There are multiple ways to construct dates with pandas. The **`to_datetime`** function is very flexible with what it accepts as a date string. Here, we simply concatenate the month and year and place a comma between the values.

In [ ]:
dates = library['Circulation Active Month'] + ', ' + library['Circulation Active Year']
dates.head()

In [ ]:
pd.to_datetime(dates.head())

In [ ]:
circulation_ad = pd.to_datetime(dates, errors='coerce')

## Use the **`insert`** method to put in new column in a specific place

In [ ]:
loc = library_new.columns.get_loc('Circulation Active Month')
loc

This happens **inplace**

In [ ]:
library_new.insert(loc, 'Circulation Active Date', circulation_ad)

In [ ]:
library.head()

In [ ]:
circulation_ad.isna().sum()

## Drop the old Circulation columns

In [ ]:
library_new = library_new.drop(columns=['Circulation Active Year', 'Circulation Active Month'])
library_new.head()

## One more set of columns - Home Library

In [ ]:
home_library_codes = pd.crosstab(library_new['Home Library Code'], library_new['Home Library Definition'])
home_library_codes.head(10)

In [ ]:
home_library_codes.sum() == home_library_codes.max()

## Not quite a one-to-one mapping 
Some codes have an **unknown description** If we drop the code column, we would be losing some information. If most of the unknowns belong to a single code, we might be able to drop it.

In [ ]:
home_library_codes['Unknown'].sum(), home_library_codes['Unknown'].max()

In [ ]:
home_library_codes['Unknown'].ne(0).sum()

There are 45 library codes that have atleast one unknown value. So, there could be value in keeping this column but for now, we will drop it to simplify our lives and make a note of this.

In [ ]:
library_new = library_new.drop(columns='Home Library Code')
library_new.head()

## Clean-up data dictionary
We have dropped several columns from our data. Let's drop them from the index of our data dictionary.

In [ ]:
library_dd_new= library_dd_new.drop(index=['Patron Type Code', 'Home Library Code', 
                                           'Circulation Active Month', 'Circulation Active Year', 
                                           'Notice Preference Code'])
library_dd_new

# We also added a column to our data dictonary
Let's append it to the bottom (and correct its placement later)

In [ ]:
dtype = library_new.dtypes['Circulation Active Date']
description = "Date the patron last checked out library materials, or last logged into the library's " \
              "subscription databases from a computer outside the library. "
count_na = library_new['Circulation Active Date'].isna().sum()
pct_na = library_new['Circulation Active Date'].isna().mean()
note = 'This column replaced both Circulation Active Month/Year'

In [ ]:
library_dd_new.loc['Circulation Active Date'] = [dtype, description, note, count_na, pct_na]

In [ ]:
library_dd_new

## Add notes for **Code** columns that were dropped

In [ ]:
library_dd_new.loc['Patron Type Definition', 'Note'] = 'Patron Code had 1-1 mapping and was subsequently dropped'
library_dd_new.loc['Home Library Definition', 'Note'] = 'Home library code had nearly 1-1 mapping except for Unknown, but was stil dropped'
library_dd_new.loc['Notice Preference Definition', 'Note'] = 'Patron Code had 1-1 mapping and was subsequently dropped'

In [ ]:
library_dd_new

## Rearrange Columns
Initial column order is not something set in stone. You can change the order to better suit your needs. Typically, categorical variables are placed before continuous variables, but you can create an arrangement that helps you analyze the data better.

In [ ]:
library_new.head()

It helps to print out the columns:

In [ ]:
library_new.columns

In [ ]:
new_col_order = ['Patron Type Definition', 'Age Range', 'Home Library Definition', 
                 'Circulation Active Date', 'Total Checkouts', 'Total Renewals',
                 'Notice Preference Definition', 'Provided Email Address',
                 'Year Patron Registered', 'Outside of County', 'Supervisor District']
len(new_col_order) == len(library_new.columns)

In [ ]:
set(new_col_order) == set(library_new.columns)

In [ ]:
library_new = library_new[new_col_order]
library_new.head()

## Use these columns to rearrange the data dictionary as well

In [ ]:
library_dd_new.loc[new_col_order]

## Tidier data
Data should now be easier to do analysis. There are other datasets that take quite a lot more work.

# Univariate Descriptive Statistics
I generally recommend by starting with simple one-dimensional data analysis once you have tidy data. The chart in the checklist sheet can help you determine what graphical/non-graphical approaches you can take based on the type of data.

## Using the describe method
The describe method is good for getting the five-number summary (min, max, median, first/third quartiles) for continuous data. It also gives the number of unique values and most common value for the categorical values.

In [ ]:
library_new.describe(include='O').T

Let's add the number of unique values to our data dictionary

In [ ]:
library_new.nunique()

In [ ]:
library_dd_new['count unique'] = library_new.nunique()
library_dd_new

## Being efficient with categorical data
Pandas has a specific data type for dealing with low-cardinality text columns

In [ ]:
library_new.memory_usage(deep=True)

In [ ]:
library_new = library_new.astype({'Patron Type Definition': 'category',
                                  'Age Range': 'category',
                                  'Home Library Definition': 'category',
                                 'Notice Preference Definition': 'category'})
library_new.dtypes

In [ ]:
library_dd_new['Data Type'] = library_new.dtypes
library_dd_new

In [ ]:
# huge decrease in memory
library_new.memory_usage(deep=True)

# Example for Categorical Data
* Use frequency counts for categorical data (non-graphical)
* Bar charts (graphical)

In [ ]:
library_new['Patron Type Definition'].value_counts(normalize=True).head()

In [ ]:
library_new['Patron Type Definition'].value_counts().plot(kind='bar')

## Categorical Outliers
Categories with low counts can be examined further.

In [ ]:
library_new['Patron Type Definition'].value_counts()

## Create an extra column to label these as outliers

In [ ]:
library_new['Patron Type Definition'].value_counts().lt(100)[lambda x: x].index

In [ ]:
patron_outliers = ['BOOKS BY MAIL', 'AT USER SENIOR', 'AT USER JUVENILE',
                   'AT USER WELCOME', 'AT USER TEEN', 'FRIENDS FOR LIFE']

In [ ]:
library_new['Patron Outliers'] = library_new['Patron Type Definition'].isin(patron_outliers)
library_new.head()

# Univariate Analysis with Continuous data
Good idea to start with describe method

In [ ]:
summary_cont = library_new.describe().T.loc[:, 'mean':]
summary_cont

## Possibly add 5 number summary to data dictionary
But, it adds quite a bit messiness

In [ ]:
library_dd_new.join(summary_cont)

## Get a sense of the distribution
Histograms are great for this

In [ ]:
library_new['Total Checkouts'].plot(kind='hist', bins=50)

Looks like there are some extreme outliers here

In [ ]:
sns.boxplot('Total Checkouts',  data=library_new)

### Choose a cut-off and examine those points later

In [ ]:
(library_new['Total Checkouts'] > 5000).mean()

## Can add these as outliers again

In [ ]:
library_new['Outlier Total Checkouts'] = library_new['Total Checkouts'] > 5000

## Filter for outliers:

In [ ]:
library_new[library_new['Outlier Total Checkouts']].head()

# Continue with checklist
* Multivariate Analysis
* Check for duplicated rows
* Bootstrap to get more samples